<a href="https://colab.research.google.com/github/borbysh/Momentum-QNG/blob/main/min_vertex_cover_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
#!pip install pennylane
#!pip install qulacs pennylane-qulacs
#!pip install seaborn
#!pip install lightning
#!pip install ipynbname

In [ ]:
sessions = [
    {"optimizer" : "QNGOptimizer",
     "params" : {"approx" : 'block-diag', "lam" : 0.5},
     "params_name" : "approx='block-diag', $\mathit{"+ chr(955) + "}$=0.5",
     "stepsizes" : [0.01,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25],
     "dev" : "default.qubit"},
    {"optimizer" :  "AdamOptimizer",
     "params" : {"beta1" : 0.9, "beta2" : 0.99, "epsilon" : 1.0e-8},
     "params_name" : "$\mathit{"+ chr(946) + "}_1$=0.9, $\mathit{"+ chr(946) + "}_2$=0.99, $\mathit{"+ chr(949) +"}$=$10^{-8}$",
     "stepsizes" : [0.01,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25],
     "dev" : "default.qubit"},
    {"optimizer" :  "MomentumOptimizer",
     "params" : {"momentum" : 0.9},
     "params_name" : "$\mathit{"+ chr(961) +"}$=0.9",
     "stepsizes" : [0.001,0.003,0.005,0.007,0.009,0.011,0.013,0.015,0.017,0.019,0.021],
     "dev" : "default.qubit"},
    {"optimizer" :  "MomentumQNGOptimizer",
     "params" : {"momentum" : 0.9, "approx" : 'block-diag', "lam" : 0.5},
     "params_name" : "$\mathit{"+ chr(961) +"}$=0.9, approx='block-diag', $\mathit{"+ chr(955) + "}$=0.5",
     "stepsizes" : [0.01,0.025,0.05,0.075,0.1,0.125,0.15,0.175,0.2,0.225,0.25],
     "dev" : "default.qubit"}
]

In [ ]:
init_count = 200
steps = 200

cost_tolerance = 0.01
cost_tolerance_count = 3



In [ ]:
session_paths = [
    ["qng",["/1","/2","/3","/4","/5","/6","/7","/8"]],
    ["adam",[""]],
    ["mom",["/1"]],
    ["momqng",["/1","/2","/3","/4","/5","/6","/7","/8"]]
]

In [ ]:
arr = [ [0]*2 for i in range(3)]
print(arr)
arr[0][0] = 1
print(arr)

In [ ]:
import glob

def read_data(session_path):
    
    stepsizes = []
    session_name = session_path[0]
    for path in session_path[1]:
        pattern = "../"+session_name+path+ "/mvc8_low_"+session_name+"_stepsizes_*.dat"
        print(pattern)
        filenames = glob.glob(pattern)
        with open(filenames[-1], "r") as file:
            vals = file.readlines()[0].split()
            for i in range(1,len(vals)):
                stepsizes += [float(vals[i])]

    total_stepsize_count = len(stepsizes)
    print(total_stepsize_count,stepsizes)
    quality = [ [0.0]*init_count for i in range(total_stepsize_count)]
    steps = [ [0.0]*init_count for i in range(total_stepsize_count)]
    start_stepsize_index = 0
    for path in session_path[1]:
        pattern = "../"+session_name+path+ "/mvc8_low_"+session_name+"_stepsizes_*.dat"
        filenames = glob.glob(pattern)
        stepsizes_count = 0
        with open(filenames[-1], "r") as file:
            stepsizes_count = len(file.readlines()[0].split())-1      
        
        pattern = "../"+session_name+path+ "/mvc8_low_"+session_name+"_quality_*.dat"
        filenames = glob.glob(pattern)
        with open(filenames[-1], "r") as file:
            lines = file.readlines()
            for iline,line in enumerate(lines):
                vals = line.split()
                for i in range(stepsizes_count):
                    quality[start_stepsize_index + i][iline] = float(vals[i+1])
        pattern = "../"+session_name+path+ "/mvc8_low_"+session_name+"_steps_*.dat"
        filenames = glob.glob(pattern)
        with open(filenames[-1], "r") as file:
            lines = file.readlines()
            for iline,line in enumerate(lines):
                vals = line.split()
                for i in range(stepsizes_count):
                    steps[start_stepsize_index + i][iline] = float(vals[i+1])
        start_stepsize_index += stepsizes_count
        
            
    
    return stepsizes, quality, steps
    

In [ ]:
stepsizes, cur_quality, cur_steps = read_data(session_paths[1])

In [ ]:
sessions_steps = []
sessions_quality = []

qng_stepsizes, qng_quality, qng_steps = read_data(session_paths[0])
sessions_quality.append(qng_quality)
sessions_steps.append(qng_steps)
sessions[0]["stepsizes"] = qng_stepsizes

adam_stepsizes, adam_quality, adam_steps = read_data(session_paths[1])
sessions_quality.append(adam_quality)
sessions_steps.append(adam_steps)
sessions[1]["stepsizes"] = adam_stepsizes

mom_stepsizes, mom_quality, mom_steps = read_data(session_paths[2])
sessions_quality.append(mom_quality)
sessions_steps.append(mom_steps)
sessions[2]["stepsizes"] = mom_stepsizes

momqng_stepsizes, momqng_quality, momqng_steps = read_data(session_paths[3])
sessions_quality.append(momqng_quality)
sessions_steps.append(momqng_steps)
sessions[3]["stepsizes"] = momqng_stepsizes
max_stepsizes = max([len(session["stepsizes"]) for session in sessions])
print(max_stepsizes)

In [ ]:
import matplotlib as mpl
from datetime import date
import ipynbname
import numpy as np
from matplotlib import pyplot as plt

mpl.rcParams['font.size'] = 18

prefix_name = ipynbname.name() + "_" #+ str(date.today()) + "_"
prefix_name = "mvc_8_"
cmap = mpl.colormaps['tab20b']
colors = cmap(np.linspace(0, 1, max_stepsizes))

for i,session in enumerate(sessions):
    fig = plt.figure(figsize=(9, 7), layout="compressed")
    axs = fig.subplots(1, 1, sharex=True, sharey=True)

    # Cumulative distributions.

    for j,stepsize in enumerate(session["stepsizes"]):
        #print(i,j,stepsize)
        axs.ecdf(sessions_quality[i][j],complementary=True, color=colors[j], label=f"$\eta$={stepsize:.4}" )

    # Label the figure.
    title = "Complementary cumulative distributions\n" + session["optimizer"]
    if(len(session["params_name"]) > 0):
        title += ", " + session["params_name"]

    fig.suptitle(title)

    axs.grid(True)
    axs.legend(fontsize=16,ncol=2)
    axs.set_xlabel("quality ratio")
    axs.set_ylabel("Probability of occurrence")
    axs.label_outer()
    axs.set_xlim(0.0, 0.5)
    plt.show()
    #fig.savefig(prefix_name + session["optimizer"]+"_ccdf_quality.png",format='png',dpi=300)
    #fig.savefig(prefix_name + session["optimizer"]+"_ccdf_quality.pdf",format='pdf',dpi=300)
    #fig.savefig(prefix_name + session["optimizer"]+"_ccdf_quality.eps",format='eps',dpi=300)

In [ ]:
import matplotlib as mpl

cmap = mpl.colormaps['tab20b']
colors = cmap(np.linspace(0, 1, max_stepsizes))

for i,session in enumerate(sessions):
    fig = plt.figure(figsize=(9, 7), layout="compressed")
    axs = fig.subplots(1, 1, sharex=True, sharey=True)

    # Cumulative distributions.

    for j,stepsize in enumerate(session["stepsizes"]):
        axs.ecdf(sessions_steps[i][j],complementary=True, color=colors[j], label=f"$\eta$={stepsize:.4}" )

    # Label the figure.
    title = "Complementary cumulative distributions\n" + session["optimizer"]
    if(len(session["params_name"]) > 0):
        title += ", " + session["params_name"]

    fig.suptitle(title)

    axs.grid(True)
    axs.legend(fontsize=16,ncol=2)
    axs.set_xlabel("steps")
    axs.set_ylabel("Probability of occurrence")
    axs.label_outer()
    axs.set_xlim(0.0, 201)
    plt.show()
    #fig.savefig(prefix_name + session["optimizer"]+"_ccdf_steps.png",format='png',dpi=300)
    #fig.savefig(prefix_name + session["optimizer"]+"_ccdf_steps.pdf",format='pdf',dpi=300)
    #fig.savefig(prefix_name + session["optimizer"]+"_ccdf_steps.eps",format='eps',dpi=300)

In [ ]:
import matplotlib as mpl
mpl.rcParams['font.size'] = 11

fig = plt.figure(figsize=(9, 7), layout="compressed")
axs = fig.subplots(2, 2, sharex=True, sharey=True)

# Cumulative distributions.
cmap = mpl.colormaps['tab20b']
colors = cmap(np.linspace(0, 1, max_stepsizes))

for j,stepsize in enumerate(sessions[0]["stepsizes"]):
    #print(j,stepsize)
    axs[0][0].ecdf(sessions_quality[0][j],complementary=True, color=colors[j], label=f"$\eta$={stepsize:.4}")
for j,stepsize in enumerate(sessions[1]["stepsizes"]):
    axs[0][1].ecdf(sessions_quality[1][j],complementary=True, color=colors[j], label=f"$\eta$={stepsize:.4}")
for j,stepsize in enumerate(sessions[2]["stepsizes"]):
    axs[1][1].ecdf(sessions_quality[2][j],complementary=True, color=colors[j], label=f"$\eta$={stepsize:.4}")
for j,stepsize in enumerate(sessions[3]["stepsizes"]):
    axs[1][0].ecdf(sessions_quality[3][j],complementary=True, color=colors[j], label=f"$\eta$={stepsize:.4}")

session_id = [0,1,3,2]
axs[0][0].text(0.07,0.07,"(a)",fontsize=16)
axs[0][1].text(0.07,0.07,"(b)",fontsize=16)
axs[1][0].text(0.07,0.07,"(c)",fontsize=16)
axs[1][1].text(0.07,0.07,"(d)",fontsize=16)
fig.suptitle("Complementary cumulative distributions")
for i in range(2):
    for j in range(2):
        axs[j][i].grid(True)
        axs[j][i].set_xlabel("quality ratio")
        axs[j][i].set_ylabel("Probability of occurrence")
        axs[j][i].label_outer()
        axs[j][i].set_xlim(0.0, 0.42)
        axs[j][i].legend(loc="upper right",fontsize=8,ncol=2)
        session = sessions[session_id[i*2+j]]
        title = session["optimizer"]
        if(len(session["params_name"]) > 0):
            title += "\n" + session["params_name"]
        axs[i][j].set_title(title)

plt.show()
#fig.savefig(prefix_name + "quality_ccdf.png",format='png',dpi=300)
fig.savefig(prefix_name + "quality_ccdf.pdf",format='pdf',dpi=300)
#fig.savefig(prefix_name + "quality_ccdf.eps",format='eps',dpi=300)

In [ ]:
import matplotlib as mpl
mpl.rcParams['font.size'] = 11

fig = plt.figure(figsize=(9, 7), layout="compressed")
axs = fig.subplots(2, 2, sharex=True, sharey=True)

# Cumulative distributions.
cmap = mpl.colormaps['tab20b']
colors = cmap(np.linspace(0, 1, max_stepsizes))

for j,stepsize in enumerate(sessions[0]["stepsizes"]):
    #print(j)
    axs[0][0].ecdf(sessions_steps[0][j],complementary=True, color=colors[j], label=f"$\eta$={stepsize:.4}" )
for j,stepsize in enumerate(sessions[1]["stepsizes"]):
    axs[0][1].ecdf(sessions_steps[1][j],complementary=True, color=colors[j], label=f"$\eta$={stepsize:.4}" )
for j,stepsize in enumerate(sessions[2]["stepsizes"]):
    axs[1][1].ecdf(sessions_steps[2][j],complementary=True, color=colors[j], label=f"$\eta$={stepsize:.4}" )
for j,stepsize in enumerate(sessions[3]["stepsizes"]):
    axs[1][0].ecdf(sessions_steps[3][j],complementary=True, color=colors[j], label=f"$\eta$={stepsize:.4}" )

session_id = [0,1,3,2]

axs[0][0].text(5,0.07,"(a)",fontsize=16)
axs[0][1].text(5,0.07,"(b)",fontsize=16)
axs[1][0].text(5,0.07,"(c)",fontsize=16)
axs[1][1].text(5,0.07,"(d)",fontsize=16)
fig.suptitle("Complementary cumulative distributions")
for i in range(2):
    for j in range(2):
        axs[j][i].grid(True)
        axs[j][i].set_xlabel("steps")
        axs[j][i].set_ylabel("Probability of occurrence")
        axs[j][i].label_outer()
        axs[j][i].set_xlim(0.0, 201)
        axs[j][i].legend(loc="upper right",fontsize=8,ncol=2)
        session = sessions[session_id[i*2+j]]
        title = session["optimizer"]
        if(len(session["params_name"]) > 0):
            title += "\n" + session["params_name"]
        axs[i][j].set_title(title)

plt.show()
#fig.savefig(prefix_name + "steps_ccdf.png",format='png',dpi=300)
fig.savefig(prefix_name + "steps_ccdf.pdf",format='pdf',dpi=300)
#fig.savefig(prefix_name + "steps_ccdf.eps",format='eps',dpi=300)

In [ ]:
#!pip install torch

In [ ]:
import matplotlib as mpl
import statistics
import math
import torch

mpl.rcParams['font.size'] = 16

cmap = mpl.colormaps['tab20b']
colors = cmap(np.linspace(0, 1, max_stepsizes))
fig = plt.figure(figsize=(9, 7), layout="compressed")
axs = fig.subplots(1, 1, sharex=True, sharey=True)
axs.text(0.0015,0.012,"(c)",fontsize=28)
format_plot = ["s","o","v","^"]
for i,session in enumerate(sessions):
    ave_session = []
    stdev_session = []
    ave_plus_stdev_session = []
    ave_minus_stdev_session = []        
    for j,stepsize in enumerate(session["stepsizes"]):
        quality = []
        for k in range(init_count):
            quality.append(float(sessions_quality[i][j][k]))
        ave_val = statistics.mean(quality)
        stdev_val = statistics.stdev(quality)
        ave_session.append(ave_val)
        if(stdev_val == 'NaN'):
            stdev_val = 0
        stdev_session.append(stdev_val)
        ave_minus_stdev_session.append(ave_val - stdev_val)
        ave_plus_stdev_session.append(ave_val + stdev_val)          
    label = session["optimizer"]
    if(len(session["params_name"]) > 0):
        label += ", " + session["params_name"]
    #axs.errorbar(session["stepsizes"],ave_session,yerr=stdev_session,elinewidth=1,label=label,capsize=3,fmt=format_plot[i],ms=7,lw=2)
    axs.plot(session["stepsizes"],ave_session,label=label,marker=format_plot[i],ms=7,lw=2)
    axs.fill_between(session["stepsizes"], ave_minus_stdev_session, ave_plus_stdev_session, alpha=0.2)

    axs.grid(True)
    axs.legend(fontsize=14)
    axs.set_xlabel(r'learning rate $\eta$',fontsize=20)
    axs.set_ylabel("Mean of quality ratio",fontsize=20)
    #axs.label_outer()
    axs.set_xlim(0.001, 1)
    axs.set_xscale('log')
    axs.set_ylim(0.0, 0.25)
plt.show()
fig.savefig(prefix_name+"ave_quality.png",format='png',dpi=300)
fig.savefig(prefix_name+"ave_quality.pdf",format='pdf',dpi=300)
#fig.savefig(prefix_name+"_ave_quality.eps",format='eps',dpi=300)

In [ ]:
import matplotlib as mpl
import statistics
import math
import torch

cmap = mpl.colormaps['tab20b']
colors = cmap(np.linspace(0, 1, 20))
fig = plt.figure(figsize=(9, 7), layout="compressed")
axs = fig.subplots(1, 1, sharex=True, sharey=True)
axs.text(0.0015,10.0,"(d)",fontsize=28)
format_plot = ["s","o","v","^"]
for i,session in enumerate(sessions):
    ave_session = []
    stdev_session = []
    ave_plus_stdev_session = []
    ave_minus_stdev_session = []        
    for j,stepsize in enumerate(session["stepsizes"]):
        arr_steps = []
        for k in range(init_count):
            arr_steps.append(float(sessions_steps[i][j][k]))
        ave_val = statistics.mean(arr_steps)
        stdev_val = statistics.stdev(arr_steps)
        ave_session.append(ave_val)
        if(stdev_val == 'NaN'):
            stdev_val = 0
        stdev_session.append(stdev_val)
        ave_minus_stdev_session.append(ave_val - stdev_val)
        ave_plus_stdev_session.append(ave_val + stdev_val)          
    label = session["optimizer"]
    if(len(session["params_name"]) > 0):
        label += ", " + session["params_name"]
    #axs.errorbar(session["stepsizes"],ave_session,yerr=stdev_session,elinewidth=1,label=label,capsize=3,fmt=format_plot[i],ms=7,lw=2)
    axs.plot(session["stepsizes"],ave_session,label=label,marker=format_plot[i],ms=7,lw=2)
    axs.fill_between(session["stepsizes"], ave_minus_stdev_session, ave_plus_stdev_session, alpha=0.2)

    axs.grid(True)
    axs.legend(fontsize=14)
    axs.set_xlabel(r'learning rate $\eta$',fontsize=20)
    axs.set_ylabel("Mean of steps",fontsize=20)
    #axs.label_outer()
    axs.set_xlim(0.001, 1)
    axs.set_xscale('log')
    axs.set_ylim(0.0, 210)
plt.show()
fig.savefig(prefix_name+"ave_steps.png",format='png',dpi=300)
fig.savefig(prefix_name+"ave_steps.pdf",format='pdf',dpi=300)
#fig.savefig(prefix_name+"_ave_steps.eps",format='eps',dpi=300)